In [26]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urltools
import time 


# In[2]:
#location="`D:\WHIN\"

def get_base_url(url):
    from urllib.parse import urlsplit
    base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url))
    

    if base_url.startswith("https"):
        base_url = base_url.replace('https://','')
    elif base_url.startswith("http"):
        base_url = base_url.replace('http://','')
        
    if base_url.endswith('/'):
        base_url = base_url.replace('/','')
    return base_url


# In[3]:


def does_page_exist(url):
    exists = 0 
    try:
        request = requests.get(url)
        if request.status_code == 200:
            exists = 1 
            
    except: 
        pass
    
    return exists


# In[4]:


# Function to extract links from a URL------------------------------------------
def href_scrapper(url):

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        a_tag = soup.find_all('a')
        img_tag = soup.find_all('img')
        link_tag = soup.find_all('link')
        link_list = list()

        for links in a_tag:
            link_list.append(links.get('href'))

        for links in link_tag:
            link_list.append(links.get('href'))

        for links in img_tag:
            link_list.append(links.get('src'))

        return link_list

    except:
        return 0



# In[5]:


# Function to create complete links---------------------------------------------
def link_former(url):
    x = url

    if x.startswith("http"):
        return x

    else:
        full_url = original_url + x
        return full_url



# In[6]:


# Function to save links to directory of python file----------------------------
def file_save(url, i):
    save_error = 0
    full_url = url
    try:
        if str(full_url).endswith(".htm"):
            name = "WebPage" + str(i) + ".htm"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".pdf"):
            name = "Pdf" + str(i) + ".pdf"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpg"):
            name = "Image" + str(i) + ".jpg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpeg"):
            name = "Image" + str(i) + ".jpeg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".png"):
            name = "Image" + str(i) + ".png"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".css"):
            name = "CSS File" + str(i) + ".css"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        else:
            name = "WebPage" + str(i) + ".html"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        return m
    except:
        save_error = save_error + 1




# In[7]:

def crawl(original_url, num_Id):
    
    # Accepting input of URL and depth----------------------------------------------
    tic = time.time()
    
    parent_list = [original_url]
    url_to_check = get_base_url(original_url)
    print(url_to_check)
    url_to_check = str(url_to_check)

    layer_stop = 1
    layer = 0


    #initializing all the requiered lists---------------------------------------------
    visited_all  = [original_url]
    visited_current_layer = []
    child_list =[]
    child_list_filtered = []
    #columns = ['Link','Parent Link', 'Layer']
    df = pd.DataFrame()


    # Main execution of scrapper----------------------------------------------------

    #looping through layers 
    while layer < layer_stop:
    
        #looping through URLs in parent-list
        for url in parent_list:
        
            #scraping the children from the parent url----------------------------
            if href_scrapper(url) != 0:
                child_list = href_scrapper(url)
    
    
            for child in child_list:
                if child != None:
                    #if child link is of the form "index.php/blahblah" and parent ends with '/'
                    #---> "parentlink/index.php/blahblah"
                    if child.startswith('/'):
                        child= str(url) + str(child)
                    
                    if url.endswith('/') and url_to_check not in child:
                        child = str(url) + str(child)
                    
                #normalize the child links-------------------------------------
                    child=urltools.normalize(child)  
                    

                    #filtering out based on 1) External 2) Repeating 3) Invalid links---------------------------
                    if url_to_check in child and child not in visited_all and does_page_exist(child)==1:
                        child_list_filtered.append(child)
                
                    #adding everthing to visited all--------------------
                    if child not in visited_all:
                        child_slash = child + '/'
                        visited_all.append(child)  
                        visited_all.append(child_slash)
                    
                    
            #adding  the visited and filtered children into the "current visited layer" ------ 
            for child_filtered in child_list_filtered:
                visited_current_layer.append(child_filtered)

            #creating a Pandas dataframe to store everything for download----------
            layer_number = [layer+1]*len(child_list_filtered)
            parent_of_child = [url]*len(child_list_filtered)

            df_child = pd.DataFrame(child_list_filtered)
            df_parent = pd.DataFrame(parent_of_child)
            df_layer = pd.DataFrame(layer_number)


            df_to_be_added = pd.concat([df_child,df_parent,df_layer], axis=1)
            df = pd.concat([df,df_to_be_added],ignore_index=True, axis = 0)
            #----------------------------------------------------------------------
        
            #emptying the child lists
            child_list = []
            child_list_filtered = []
        
        #condition to stop filtering-----------------------------------------------
        if not visited_current_layer :
            layer_stop = layer_stop 
        else:
            layer_stop += 1

    
        #child layer is now parent layer--------------------------------------------
        parent_list = []
    
        #we only dont add .png, .jpg , .pdf to the new parent layer 
        for visited_current in visited_current_layer: 
            print(visited_current)
            if(not visited_current.endswith('.png') and not visited_current.endswith('.jpg') and not  visited_current.endswith('.pdf')):
                parent_list.append(visited_current)
            
            
        #displaying the links in different layers----------------------------------
        #print("Links in LAYER:" + str(layer+1))
        print("No of links = " + str(len(visited_current_layer)))
        #print(visited_current_layer)
        print("\n")
        visited_current_layer = [] 
        #updating the layer number
        layer +=1
    df.to_csv('D:\WHIN\''+num_Id+'.csv', sep='\t', encoding='utf-8')
    return none


In [29]:
original_url = "https://www.rowetruck.com"
crawl(original_url,1)

www.rowetruck.com
https://www.rowetruck.com/deals
https://www.rowetruck.com/
https://www.rowetruck.com/products/dump-bodies/contractor
https://www.rowetruck.com/products/dump-bodies/demolition
https://www.rowetruck.com/products/dump-bodies/light-weight
https://www.rowetruck.com/products/dump-bodies/post-side
https://www.rowetruck.com/products/dump-bodies/sba1000
https://www.rowetruck.com/products/dump-bodies/smoothside
https://www.rowetruck.com/products/dump-bodies/specialty
https://www.rowetruck.com/products/dump-bodies/stainless
https://www.rowetruck.com/products/parts/body-props
https://www.rowetruck.com/products/parts/bottom-tailgate-latch
https://www.rowetruck.com/products/parts/coal-chute
https://www.rowetruck.com/products/parts/del1229-hoistpto-control
https://www.rowetruck.com/products/parts/high-lift-tailgate-cylinder
https://www.rowetruck.com/products/parts/hoist-trunion-wpin
https://www.rowetruck.com/products/parts/hydraulic-tank
https://www.rowetruck.com/products/parts/hydr

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://www.rowetruck.com/plugins/atg/showcase/assets/css/plugin.css
https://www.rowetruck.com//deals
https://www.rowetruck.com//products/dump-bodies
https://www.rowetruck.com//products/suspensions
https://www.rowetruck.com//parts
https://www.rowetruck.com//service
https://www.rowetruck.com//storage/app/media/an-american-body-logo.png
https://www.rowetruck.com//storage/app/media/home-features/ASC8.png
https://www.rowetruck.com//storage/app/media/20180502_103717_resized.jpg
https://www.rowetruck.com//storage/app/media/25k non steer.jpg
https://www.rowetruck.com//storage/app/media/home-features/parts-2to1-min.jpg
https://www.rowetruck.com//storage/app/media/home-features/service-2to1-min.jpg
https://www.rowetruck.com/storage/app/uploads/public/592/71c/3c3/59271c3c39d40994558927.jpg
https://www.rowetruck.com/storage/app/uploads/public/592/71e/4c6/59271e4c661d9895869259.jpg
https://www.rowetruck.com/storage/app/uploads/public/592/71e/751/59271e751f09a593643307.jpg
https://www.rowetruck.com

https://www.rowetruck.com/products/parts?page=1
https://www.rowetruck.com/products/parts?page=2
https://www.rowetruck.com/products/parts?page=3
https://www.rowetruck.com/products/dump-bodies?page=1/storage/app/media/an-american-body-logo.png
https://www.rowetruck.com/products/dump-bodies?page=1/storage/app/media/home-features/ASC8.png
https://www.rowetruck.com/products/dump-bodies?page=2/storage/app/media/an-american-body-logo.png
https://www.rowetruck.com/products/dump-bodies?page=2/storage/app/media/home-features/ASC8.png
No of links = 7


https://www.rowetruck.com/products/parts?page=1/storage/app/media/an-american-body-logo.png
https://www.rowetruck.com/products/parts?page=1/storage/app/media/home-features/ASC8.png
https://www.rowetruck.com/products/parts?page=2/storage/app/media/an-american-body-logo.png
https://www.rowetruck.com/products/parts?page=2/storage/app/media/home-features/ASC8.png
https://www.rowetruck.com/products/parts?page=3/storage/app/media/an-american-body-logo.pn

TypeError: must be str, not int

In [2]:
import requests

page = requests.get("https://docs.python.org/3/howto/urllib2.html")
from bs4 import BeautifulSoup
from bs4.element import Comment
soup = BeautifulSoup(page.content, 'html.parser')
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
p_tag=soup.find_all('p')
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
web_text = []


In [20]:
import json
import urllib
def scrape(df, num_Id):
    TempD={}
    for index,item in df.iterrows():
        try:      
            html = urllib.request.urlopen(item.iloc[1]).read()
            web_text.append(text_from_html(html))
            print(text_from_html(html))
            TempD[item.iloc[0]]=text_from_html(html)
        except: 
            pass
    with open(num_Id+'.json', 'w') as outfile:  
        json.dump(TempD, outfile)
    return 1      

              Deals!  Sign Up            Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact              Our state of the art facilities and unrivaled craftsmanship set us apart. See why Rowe Truck Equipment is the 

              Deals!  Sign Up            Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact              Our state of the art facilities and unrivaled craftsmanship set us apart. See why Rowe Truck Equipment is the 

              Deals!  Sign Up            Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact              Our state of the art facilities and unrivaled craftsmanship set us apart. See why Rowe Truck Equipment is the 

              Deals!  Sign Up            Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact              Our state of the art facilities and unrivaled craftsmanship set us apart. See why Rowe Truck Equipment is the 

              Deals!  Sign Up            Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact              Our state of the art facilities and unrivaled craftsmanship set us apart. See why Rowe Truck Equipment is the 

...

                   Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact                     Contractor  The PLD2000 Platform Lift Dump Body is a popular choice for many operators who need a versatile option when worki

                   Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact                     Contractor  The PLD2000 Platform Lift Dump Body is a popular choice for many operators who need a versatile option when worki

                   Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact                     Specialty  Rowe Truck doesn’t just manufacture dump bodies. We fabricate and install many unique units. Roll-Off bodies, Swap

                   Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact                     Tailgate Latch Linkage  $33.00 ea  Learn More       Upper Cylinder Pin  $33.00  Learn More                 Upper Hinge  $136.

TypeError: unsupported operand type(s) for +: 'builtin_function_or_method' and 'str'

In [19]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
 
df = pd.read_excel('D:\WHIN\Final Final List of Companies.xlsx', sheetname='Contacted Companies')
for row, index in df.rowiterator():
    crawl(row.iloc[2],row.iloc[0])


{'https://www.rowetruck.com/deals': '              Deals!  Sign Up            Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact              Our state of the art facilities and unrivaled craftsmanship set us apart.

In [14]:
for row, index in df.rowiterator():
    num_Id=row.iloc[0]
    original_url=row.iloc[2]
    try:
        websites= pd.read_csv(row.iloc[0]+'.csv')
    except:
        websites=crawl(row.iloc[2],row.iloc[0])
    flag=scrape(websites,num_Id )
    
    

              Deals!  Sign Up            Toggle navigation     Menu           Products         Dump Bodies    Contractor    Demolition    Light Weight    Post Side    SBA1000    Smoothside    Specialty    Stainless        Dump Body Parts    Body Props    Bottom Tailgate Latch    Coal Chute    DEL1229 Hoist/PTO Control    High-lift Tailgate Cylinder    Hoist Trunion w/Pin    Hydraulic Tank    Hydraulic Tank Cap    Lower Cylinder Pin    Quad Axle Control Tower    Rear Body Hinge    Tailgate Cylinder    Tailgate Latch Linkage    Upper Cylinder Pin    Upper Hinge    Upper Hinge Base    Upper Hinge Pin    Upper Hoist Trunion w/Pin         Suspensions    13.2K & 20K Truck Steerable    15K & 20K Trailer Steerable    20K Crank Axle Non-Steerable    25K Trailer Non-Steerable    25K Truck Non-Steerable    Application Specific         Parts    Service    Driveline    Contact              Our state of the art facilities and unrivaled craftsmanship set us apart. See why Rowe Truck Equipment is the 